# Matrix vector multiplication in SHEEP
Since we can use "slots", i.e. vectors of input values, and many libraries support SIMD operations (i.e. operating on many slots is just as quick as operating on single values), we can easily do component-wise multiplication.  However, as with vector dot products, we generally need to also sum over slots, which can be done using a sequence of "ROTATE" and "ADD" operations.

In [53]:
import os
if "SHEEP_HOME" in os.environ.keys():
  SHEEP_HOME = os.environ["SHEEP_HOME"]
else:
  SHEEP_HOME = os.path.join(os.environ["HOME"],"SHEEP","frontend")
import sys
sys.path.append(SHEEP_HOME)

from pysheep import sheep_client

## Multiplying a 3x3 matrix with a 3-vector


Lets do the following calculation:
\begin{equation*}
\begin{vmatrix}
1, 2 , 3 \\
4 , 5 , 6 \\
7 , 8 , 9
\end{vmatrix}
\begin{vmatrix}
1 \\
2 \\
3
\end{vmatrix}
\end{equation*}

which should give us the answer ```(14, 32, 50)```


### a) The straightforward way

Essentially the each element of the output vector will be the dot product of the corresponding row of the matrix with the vector.  We can therefore do this in the same way as was demonstrated in the vector_dot_product notebook, i.e. component-wise multiplication followed by a sequence of rotations and additions.

The circuit will look like:

In [65]:
circuit = """
INPUTS mrow_0 mrow_1 mrow_2 vec mask
CONST_INPUTS rotate_by_minus1 rotate_by_plus1
OUTPUTS output_vec sum_0 sum_1 sum_2 prod_00 prod_02 prod_02
# dot product of the first row with the vector
mrow_0 vec MULTIPLY prod_00
prod_00 rotate_by_minus1 ROTATE prod_01
prod_00 prod_01 ADD sum_00
prod_01 rotate_by_minus1 ROTATE prod_02
sum_00 prod_02 ADD sum_0
# dot product of the second row with the vector
mrow_1 vec MULTIPLY prod_10
prod_10 rotate_by_minus1 ROTATE prod_11
prod_10 prod_11 ADD sum_10
prod_11 rotate_by_minus1 ROTATE prod_12
sum_10 prod_12 ADD sum_1
# dot product of the third row with the vector
mrow_2 vec MULTIPLY prod_20
prod_20 rotate_by_minus1 ROTATE prod_21
prod_20 prod_21 ADD sum_20
prod_21 rotate_by_minus1 ROTATE prod_22
sum_20 prod_22 ADD sum_2
# now we have three vectors, sum_0, sum_1, and sum_2, where the first element is 
# the dot product of that row.  We need to isolate just this element, using mask, which is [1,0,0]
sum_0 mask MULTIPLY msum_0
sum_1 mask MULTIPLY msum_10
msum_10 rotate_by_plus1 ROTATE msum_1
sum_2 mask MULTIPLY msum_20
msum_20 rotate_by_plus1 ROTATE msum_2
# now we should have three vectors with one non-zero element each in the right place - need to sum them
msum_0 msum_1 ADD out_01
out_01 msum_2 ADD output_vec
"""

In [66]:
sheep_client.new_job()
sheep_client.set_context("HElib_Fp")
sheep_client.set_input_type("int8_t")
#sheep_client.set_parameters({"NumSlots": 3})
#sheep_client.set_parameters({"BaseParamSet": 2})
sheep_client.set_circuit_text(circuit)
sheep_client.get_inputs()

{'content': ['mrow_0', 'mrow_1', 'mrow_2', 'vec', 'mask'], 'status_code': 200}

In [67]:
sheep_client.get_const_inputs()

{'content': ['rotate_by_minus1', 'rotate_by_plus1'], 'status_code': 200}

In [68]:
sheep_client.set_inputs({"mrow_0": [1,2,3], "mrow_1": [4,5,6], "mrow_2": [7,8,9], "vec": [1,2,3], "mask": [1,0,0]})

{'content': '', 'status_code': 200}

In [69]:
sheep_client.set_const_inputs({"rotate_by_minus1": -1, "rotate_by_plus1": 1})

{'content': '', 'status_code': 200}

In [70]:
sheep_client.run_job()

{'content': '', 'status_code': 200}

In [71]:
sheep_client.get_results()

{'content': {'cleartext check': {'is_correct': False},
  'outputs': {'output_vec': ['68,-60,40'],
   'prod_00': ['7,0,0'],
   'prod_02': ['0,0,7'],
   'sum_0': ['7,0,7'],
   'sum_1': ['50,43,34'],
   'sum_2': ['122,94,82']},
  'timings': {'decryption': '3107.100000',
   'encryption': '4994.600000',
   'evaluation': '53634.800000',
   'output_vec': '2057.500000',
   'prod_00': '2038.600000',
   'prod_02': '86.000000',
   'sum_0': '4585.200000',
   'sum_1': '2035.800000',
   'sum_2': '4408.000000'}},
 'status_code': 200}

## Doing the same calculation without the ROTATEs.
To minimize the number of intra-slot operations, we can instead express the first three inputs as diagonal strips of the matrix.

In this case, the circuit will look like:

In [89]:
new_circuit = """
INPUTS mstrip_0 mstrip_1 mstrip_2 vec mask_0 mask_1 mask_2
OUPUTS output_vec
mstrip_0 vec MULTIPLY prod_0
mstrip_1 vec MULTIPLY prod_1
mstrip_2 vec MULTIPLY prod_2
prod_0 prod_1 ADD sum_0
sum_0 prod_2 ADD sum
sum mask_0 MULTIPLY el_0
sum mask_1 MULTIPLY el_1
sum mask_2 MULTIPLY el_2
el_0 el_1 ADD newsum_01
newsum_01 el_2 ADD output_vec
"""

In [90]:
sheep_client.new_job()
sheep_client.set_context("Clear")
sheep_client.set_input_type("int8_t")
sheep_client.set_circuit_text(new_circuit)


{'content': '', 'status_code': 200}

We can set some of the inputs similarly to above - the "mask" ones are [1,0,0], [0,1,0], [0,0,1] respectively

In [91]:
input_vals = {"vec": [1,2,3],"mask_0":[1,0,0],"mask_1": [0,1,0],"mask_2": [0,0,1]}

So now we have to set the remaining inputs to be diagonal strips of the matrix:

In [92]:
input_vals["mstrip_0"] = [1,5,9]
input_vals["mstrip_1"] = [2,6,7]
input_vals["mstrip_2"] = [3,4,8]
sheep_client.set_inputs(input_vals)

{'content': '', 'status_code': 200}

In [93]:
sheep_client.run_job()

{'content': 'Could not run evaluation', 'status_code': 500}

In [94]:
sheep_client.get_results()

{'content': {'cleartext check': {'is_correct': False},
  'outputs': {},
  'timings': {}},
 'status_code': 200}